<a href="https://colab.research.google.com/github/anvy87/AlgoTrading/blob/master/goldSilverRatio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install kiteconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.2 MB/s eta 0:00:00


In [3]:
from kiteconnect import KiteConnect
import pandas as pd
import numpy as np
import logging
import os
import time
import datetime as dt
api_key = "1ll4p8ziqlk8hh7g"
api_secret = "x59r5u1vd25annfpgts22wv8ykfueirl"
kite = KiteConnect(api_key=api_key)
print(kite.login_url()) #use this url to manually login and authorize yourself

https://kite.zerodha.com/connect/login?api_key=1ll4p8ziqlk8hh7g&v=3


In [5]:
#generate trading session
request_token = "iWCyTL8kYOStwmc3J6JMppZaTZQg9xIl" #Extract request token from the redirect url obtained after you authorize yourself by loggin in
data = kite.generate_session(request_token, api_secret=api_secret)
#create kite trading object
kite.set_access_token(data["access_token"])

#get dump of all NSE instruments
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv("NSE_Instruments.csv",index=False)

# get dump of all MCX instruments
instrument_dump_mcx = kite.instruments("MCX")
instrument_df_mcx = pd.DataFrame(instrument_dump_mcx)
instrument_df_mcx.to_csv("MCX_Instruments.csv",index=False)


def instrumentLookup(instrument_df_nse, instrument_df_mcx, symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        # Try to find the instrument in NSE
        token_nse = instrument_df_nse[instrument_df_nse.tradingsymbol == symbol].instrument_token.values
        if token_nse.size > 0:
            return token_nse[0]
        # If not found in NSE, try to find in MCX
        token_mcx = instrument_df_mcx[instrument_df_mcx.tradingsymbol == symbol].instrument_token.values
        if token_mcx.size > 0:
            return token_mcx[0]
        return -1 # Return -1 if not found in either
    except Exception as e:
        print(f"Error looking up instrument {symbol}: {e}")
        return -1

def fetchOHLC(ticker,interval,duration):
  """extracts historcial data and outputs in the form of dataframe"""
  instrument = instrumentLookup(instrument_df, instrument_df_mcx, ticker)
  if instrument != -1:
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration),dt.date.today(),interval))
    # data1 = pd.DataFrame(kite.historical_data(instrument,dt.date.today(),dt.date.today(),interval))
    # print(data1.head())
    data.set_index("date",inplace=True)
    return data
  else:
    print(f"Instrument {ticker} not found.")
    return pd.DataFrame() # Return empty DataFrame if instrument not found

In [10]:
gold_oct = fetchOHLC("MCXGOLDEX", "5minute", 1)
print(f"gold_oct is {gold_oct}")
silver_oct = fetchOHLC("MCXSILVDEX", "5minute", 1)
print(f"silver_oct is {silver_oct}")

gold_oct is                                open      high       low     close  volume
date                                                                     
2025-09-08 09:15:00+05:30  30146.82  30156.38  30145.97  30156.10       0
2025-09-08 09:20:00+05:30  30156.38  30168.76  30156.38  30167.91       0
2025-09-08 09:25:00+05:30  30162.29  30171.57  30159.19  30166.23       0
2025-09-08 09:30:00+05:30  30162.01  30165.10  30128.25  30152.72       0
2025-09-08 09:35:00+05:30  30156.10  30158.91  30150.47  30158.63       0
...                             ...       ...       ...       ...     ...
2025-09-08 15:05:00+05:30  30364.27  30368.20  30358.64  30365.95       0
2025-09-08 15:10:00+05:30  30367.08  30381.14  30367.08  30380.02       0
2025-09-08 15:15:00+05:30  30376.93  30381.14  30374.67  30381.14       0
2025-09-08 15:20:00+05:30  30383.40  30383.96  30372.99  30382.55       0
2025-09-08 15:25:00+05:30  30383.96  30390.99  30382.27  30390.99       0

[75 rows x 5 columns]
sil

In [7]:
# get dump of all MCX instruments
instrument_dump_mcx = kite.instruments("MCX")
instrument_df_mcx = pd.DataFrame(instrument_dump_mcx)
instrument_df_mcx.to_csv("MCX_Instruments.csv",index=False)